In [1]:
# -*- coding:utf-8 -*-
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.stattools as ta
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import os

In [2]:
path='C:/Users/dsquant06/Desktop/test/month.csv'
dependent=['premium']
independent=['MKT']
#第二步需要用r-rf-beta*MKT求得回归方程的左侧
independent0=['roic_ttm', 'size', 'book_to_price_ratio', 'beta','MKT']#这一项是接下来回归的右侧的几个因子

In [3]:
#表格规整
def messup(df,path):
    for i in range(10):
        if 'Unnamed: 0' in df.columns.values:
            df.drop(columns='Unnamed: 0',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0列')
        elif 'Unnamed: 0.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1列')
        elif 'Unnamed: 0.1.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1.1列')
        elif 'Unnamed: 0.1.1.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1.1.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1.1.1列')
        elif 'Unnamed: 0.1.1.1.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1.1.1.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1.1.1.1列')
    if (os.path.exists(path)):
        os.remove(path)

In [4]:
df=pd.read_csv(path)

In [5]:
x=df[independent]
y=df[dependent]

In [6]:
#model=sm.OLS(pd.DataFrame(df.loc[:,['open']].values.T).values[0],pd.DataFrame(df.loc[:,['t']].values.T).values[0])
#首先将premium作为因变量，将MKT作为自变量，做回归所得到自变量前的系数为所求的beta_buffet
x=sm.add_constant(x)
reg=sm.OLS(y,x).fit()

In [7]:
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                premium   R-squared:                       0.575
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     123.0
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           1.38e-18
Time:                        14:58:28   Log-Likelihood:                 160.96
No. Observations:                  93   AIC:                            -317.9
Df Residuals:                      91   BIC:                            -312.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0124      0.004      2.760      0.0

In [8]:
#可以看出beta_buffet为0.5593,且结果是显著的
reg.params

const    0.012406
MKT      0.559342
dtype: float64

In [9]:

x1=df[independent0]
y1=df.apply(lambda x: x['premium']-(reg.params[independent])*(x['MKT']), axis=1)#这一项是r-rf-beta*MKT左侧的值

In [10]:
x1=sm.add_constant(x1)
reg1=sm.OLS(y1,x1).fit()

In [11]:
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                    MKT   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     4.466
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           0.000556
Time:                        14:58:29   Log-Likelihood:                 173.57
No. Observations:                  93   AIC:                            -333.1
Df Residuals:                      86   BIC:                            -315.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.0120    

In [12]:
#接下来需要求得rA
#如果原来的表中有以前所得到的回归，那么删除这一列
if 'ra' in df.columns.values:
    print('已删除原有的rA值')
    df.drop(columns='ra',axis=1,inplace=True)

In [13]:
#根据各个因子的系数计算rA
for i in independent0:
    df['ra']=df.apply(lambda x: (reg1.params[i])*(x[i]), axis=1)#生成ractive

In [14]:
#规整表格，输出rA的值
messup(df,path)
df.to_csv(path)

原文件删除成功


In [15]:
# if (os.path.exists(path_regression)):
#     os.remove(path_regression)
#     print("原文件删除成功")
# else:
#     print("原文件不存在，创建新的原文件")

# reg.params.to_csv(path_regression)
# df=pd.read_csv(path_regression,names=['param','coe'])


In [16]:
#设定年化率4%，在此情况下选用月收益率作为无风险利率
r=pow(1.04,12)-1

In [17]:
#接下来需要求得rA
#如果原来的表中有以前所得到的回归，那么删除这一列
if 'style' in df.columns.values:
    print('已删除原有的style值')
    df.drop(columns='style',axis=1,inplace=True)

In [18]:
#最后计算r_Buffetstyle
df['style']=df.apply(lambda x: r + (x['MKT'])*(reg.params[independent])+x['ra'], axis=1)#生成ractive


In [19]:
messup(df,path)
df.to_csv(path)

原文件删除成功


In [20]:
#最后表格里面得到的style代表着巴菲特投资组合的风格，
#即"matches Berkshire’s beta, idiosyncratic volatility, total volatility, and relative active loadings"